In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))  
from utils.load_config import load_cfg
from utils.minio_client import MinioClient
import pandas as pd
from io import StringIO

In [2]:
config = load_cfg('../config/config.yaml')
minio_config = config['minio']
processed_folder_path = config['dataset']['processed_path']

# Get sales data from minio

In [ ]:
minio_client = MinioClient(minio_config)

sales = minio_client.get_file(
    bucket_name=minio_config['bucket_name'],
    object_name='sales'
)

sales_df = pd.read_csv(StringIO(sales))
print(sales_df.head())

Bucket "m5-forecasting-bucket" already exists.
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  d_1908  \
0       CA    0    0    0    0  ...       1       3       0       1       1   
1       CA    0    0    0    0  ...       0       0       0       0       0   
2       CA    0    0    0    0  ...       2       1       2       1       1   
3       CA    0    0    0    0  ...       1       0       5       4       1   
4       CA    0    0    0    0  ...       2       1       1       0       1  

# Get sell price data

In [9]:
sell_price_df = pd.read_parquet(f'../{processed_folder_path}/sell_prices.parquet')
sell_price_df = sell_price_df[['store_id', 'item_id']].drop_duplicates().reset_index(drop=True)
sell_price_df['id'] = sell_price_df['item_id'].astype(str) + \
                    '_' + sell_price_df['store_id'].astype(str) + \
                    '_' + 'validation'

sell_price_df = sell_price_df[['id']].drop_duplicates().reset_index(drop=True)
sell_price_df

,id
0,HOBBIES_1_001_CA_1_validation
1,HOBBIES_1_002_CA_1_validation
2,HOBBIES_1_003_CA_1_validation
3,HOBBIES_1_004_CA_1_validation
4,HOBBIES_1_005_CA_1_validation
...,...
416,HOBBIES_1_037_WI_3_validation
417,HOBBIES_1_038_WI_3_validation
418,HOBBIES_1_039_WI_3_validation
419,HOBBIES_1_040_WI_3_validation


# Filter sales data by id from sell price data

In [10]:
# filter by id from sell_price_df
filtered_sales_df = sales_df[sales_df['id'].isin(sell_price_df['id'])]
len(filtered_sales_df)

421

In [11]:
filtered_sales_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [ ]:
processed_sales_df = filtered_sales_df.copy()
processed_sales_df['store_in_state'] = processed_sales_df['store_id'].map(lambda x: x.split('_')[1])
processed_sales_df['subcategory'] = processed_sales_df['dept_id'].map(lambda x: x.split('_')[1])
processed_sales_df['item_number'] = processed_sales_df['item_id'].map(lambda x: x.split('_')[2])
processed_sales_df = processed_sales_df.rename(columns={'cat_id': 'category'})
processed_sales_df = processed_sales_df.drop(columns='id')

processed_sales_df.head()

,item_id,dept_id,category,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,store_in_state,subcategory,item_number
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,1,1,3,0,1,1,1,1,001
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,002
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,003
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,4,1,0,1,3,7,2,1,1,004
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,0,1,1,2,2,2,4,1,1,005


In [34]:
processed_sales_long_df = processed_sales_df.melt(
    id_vars=['item_id', 'dept_id', 'category', 'store_id', 'state_id', 'store_in_state', 'subcategory', 'item_number'],
    var_name='day',
    value_name='sales'
)

processed_sales_long_df.head()

,item_id,dept_id,category,store_id,state_id,store_in_state,subcategory,item_number,day,sales
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,1,001,d_1,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,1,002,d_1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,1,003,d_1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,1,004,d_1,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,1,005,d_1,0


In [29]:
len(processed_sales_long_df)

805373

# Get calendar data

In [26]:
calendar = minio_client.get_file(
    bucket_name=minio_config['bucket_name'],
    object_name='calendar'

)

calendar_df = pd.read_csv(StringIO(calendar))

In [35]:
# replace day with date
mapping_d_to_date_df = calendar_df.rename(columns={'d': 'day'})
mapping_d_to_date_df = mapping_d_to_date_df[['day', 'date']].drop_duplicates().reset_index(drop=True)
processed_sales_long_df['date'] = processed_sales_long_df['day'].map(mapping_d_to_date_df.set_index('day')['date'])
processed_sales_long_df = processed_sales_long_df.drop(columns='day')
processed_sales_long_df['wm_yr_wk'] = processed_sales_long_df['date'].map(calendar_df.set_index('date')['wm_yr_wk'])
processed_sales_long_df.head()

,item_id,dept_id,category,store_id,state_id,store_in_state,subcategory,item_number,sales,date,wm_yr_wk
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,1,001,0,2011-01-29,11101
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,1,002,0,2011-01-29,11101
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,1,003,0,2011-01-29,11101
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,1,004,0,2011-01-29,11101
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,1,005,0,2011-01-29,11101


In [36]:
processed_sales_long_df.to_parquet(f'../{processed_folder_path}/sales.parquet', index=False)